# Debugging Jupyter Notebook for OCTolyzer's loading in of `.vol` files

This notebook copies the step-by-step process of OCTolyzer's `utils.load_volfile` function which loads in a `.vol` file, and should provide the end-user with the means of debugging the pipeline by testing each step individually.

### Add OCTolyzer to system paths to permit access to analysis files

In [ ]:
import sys
sys.path.append('/Users/s1522100/Documents/OCTolyzer')
from octolyzer import utils

import eyepy
from eyepy.core import utils as eyepy_utils
from eyepy.io.he import vol_reader
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from octolyzer.measure.bscan.thickness_maps import grid
from skimage import segmentation

### Define file to be loaded and default parameters, the `vol_path` is the only variable to change

In [ ]:
# vol_path = "../demo/input/Peripapillary_1.vol"
# vol_path = "../demo/input/Ppole_1.vol"
# vol_path = "../demo/input/Linescan_1.vol"
vol_path = "../demo/input/Radial_1.vol"

logging = []
verbose = True
preprocess = True
custom_maps = ['ILM_OPL']

In [ ]:
SEG_MAPPING = {
    'ILM': 0,
    'BM': 1,
    'RNFL': 2,
    'GCL': 3,
    'IPL': 4,
    'INL': 5,
    'OPL': 6,
    'ONL': 7,
    'ELM': 8,
    'IOS': 9,
    'OPT': 10,
    'CHO': 11,
    'VIT': 12,
    'ANT': 13,
    'PR1': 14,
    'PR2': 15,
    'RPE': 16,
}

### Load image data and metadata using EyePy and determine scan pattern

In [ ]:
fname = os.path.split(vol_path)[1]
msg = f"Reading file {fname}..."
logging.append(msg)
if verbose:
    print(msg)

# Catch whether .vol file is a peripapillary or macular scan. Other locations, i.e. radial "star-shaped" scans
# currently not supported.
scan_type = "Macular"
radial = False
eyepy_import = False
try: 
    voldata = eyepy.import_heyex_vol(vol_path)
    eyepy_import = True

    # pixel data
    bscan_data = voldata.data / 255
    N_scans, M, N = bscan_data.shape
    fovea_slice_num = N_scans // 2
    
except ValueError as msg:
    if len(msg.args) > 0 and msg.args[0] == "The EyeVolume object does not support scan pattern 2 (one Circular B-scan).":
        voldata = vol_reader.HeVolReader(vol_path)
        scan_type = "Peripapillary"

        # pixel data
        pixel_data = voldata.parsed_file.bscans[0].data
        bscan_data = (eyepy_utils.from_vol_intensity(pixel_data.copy()) / 255).reshape(1,*pixel_data.shape)
        N_scans, M, N = bscan_data.shape
        fovea_slice_num = None

    elif len(msg.args) > 0 and msg.args[0] == "The EyeVolume object does not support scan pattern 5 (Radial scan - star pattern).":
        voldata = vol_reader.HeVolReader(vol_path)
        radial = True

        # pixel data
        pixel_data = [arr.data for arr in voldata.parsed_file.bscans]
        bscan_data = np.asarray([eyepy_utils.from_vol_intensity(arr.copy()) / 255 for arr in pixel_data])
        N_scans, M, N = bscan_data.shape
        fovea_slice_num = N_scans // 2
        
    else:
        logging.append(msg)
        raise msg

In [ ]:
# slo data and metadata
slo = voldata.localizer.data.astype(float) / 255
slo_N = slo.shape[0]
slo_metadict = voldata.localizer.meta.as_dict()
slo_metadict["slo_resolution_px"] = slo_N
slo_metadict["field_of_view_mm"] = slo_metadict["scale_x"] * slo_N

# bscan metadata
vol_metadata = voldata.meta.as_dict()
eye = vol_metadata["laterality"]
scale_z, scale_x, scale_y = vol_metadata["scale_z"], vol_metadata["scale_x"], vol_metadata["scale_y"]
bscan_meta = vol_metadata["bscan_meta"]

# Detect scan pattern
if scan_type == "Peripapillary":
    bscan_type = scan_type
    msg = f"Loaded a peripapillary (circular) B-scan."
    logging.append(msg)
    if verbose:
        print(msg)
elif scan_type == "Macular" and scale_z != 0:
    if radial == 0:
        bscan_type = "Ppole"
        msg = f"Loaded a posterior pole scan with {N_scans} B-scans."
    else:
        bscan_type = "Radial"
        msg = f"Loaded a radial scan with {N_scans} B-scans."
    logging.append(msg)
    if verbose:
        print(msg)
else:
    stp = bscan_meta[0]["start_pos"][0]
    enp = bscan_meta[0]["end_pos"][1]
    if np.allclose(stp,0,atol=1e-3):
        bscan_type = "H-line"
    elif np.allclose(enp,0,atol=1e-3):
        bscan_type = "V-line"
    else:
        bscan_type = "AV-line"
    msg = f"Loaded a single {bscan_type} B-scan."
    logging.append(msg)
    if verbose:
        print(msg)

### Preprocess B-scans to brighten and remove superficial retinal vessel shadowing in choroid

In [ ]:
# Optional to try compensate for vessel shadows and brighten B-scans for improved
# choroid visualisation. 
# When bscan_type is "AV-line", we do not compensate for shadows
if preprocess:
    if bscan_type != "AV-line":
        msg = "Preprocessing by compensating for vessel shadows and brightening choroid..."
        bscan_data = np.array([utils.normalise_brightness(utils.shadow_compensate(img)) for img in bscan_data])
    elif bscan_type == "AV-line":
        msg = "AV-line scans are not shadow-compensated and left raw for further processing."
    logging.append(msg)
    if verbose:
        print(msg)

### Load in retinal layer segmentations

In [ ]:
# retinal layers
retinal_layer_raw = voldata.layers
N_rlayers = len(retinal_layer_raw)
if N_rlayers == 2:
    msg = ".vol file only has ILM and BM layer segmentations."
elif N_rlayers == 17:
    msg = ".vol file contains all retinal layer segmentations."
else:
    msg = ".vol file contains ILM and BM, but fewer than all inner retinal layer segmentations."
logging.append(msg)
if verbose:
    print(msg)

### Organise retinal layer segmentations depending on scan pattern

In [ ]:
# Collect all available retinal layer keys
msg = "Processing retinal layer segmentations..."
logging.append(msg)
if verbose:
    print(msg)
x_grid_all = np.repeat(np.arange(N).reshape(-1,1), N_scans, axis=1).T

In [ ]:
# Collect all available retinal layer keys
msg = "Processing retinal layer segmentations..."
logging.append(msg)
if verbose:
    print(msg)
x_grid_all = np.repeat(np.arange(N).reshape(-1,1), N_scans, axis=1).T

# Dealing with retinal layer segmentations using primitive loader from EyePy
if not eyepy_import:
    
    # Collect retinal layer segmentations based on .vol mapping from EyePy
    retinal_layers = {}
    for name, i in SEG_MAPPING.items():
        msg = None
        if i >= retinal_layer_raw.shape[0]:
            msg = 'The volume contains less layers than expected. The naming might not be correct.'
            break
        retinal_layers[name] = retinal_layer_raw[i]

# Dealing with retinal layer segmentations loaded and organised from EyePy
else:
    retinal_layers = {name:val.data for name,val in retinal_layer_raw.items()}
        
# Create pairwise retinal layers
layer_keys = []
for key in retinal_layers.keys():
    if not np.all(np.isnan(retinal_layers[key])):
        layer_keys.append(key) 
layer_keys = layer_keys[:1] + layer_keys[2:] + ["BM"]
N_rlayers = len(layer_keys)
layer_key_pairwise = [f"{key1}_{key2}" for key1,key2 in zip(layer_keys[:-1], layer_keys[1:])]

# By default we always provide the whole retina
layer_key_pairwise.append("ILM_BM")

# If macular scan, allow custom retinal layers to be created
if bscan_type != 'Peripapillary':

    # If ELM traces exist, then we will provide measurements for inner and outer retina. 
    if 'ELM' in layer_keys:
        custom_maps += ["ILM_ELM", "ELM_BM"]
    custom_maps = list(set(custom_maps))

    # Add custom retinal layers if they exist
    if N_rlayers > 2 and len(custom_maps) > 0:
        for key_pair in custom_maps:
            layer_key_pairwise.append(key_pair)

In [ ]:
# Collect retinal layer segmentations
layer_pairwise = {}
for key in layer_key_pairwise:
    key1, key2 = key.split("_")
    lyr1 = np.concatenate([x_grid_all[...,np.newaxis],
                        retinal_layers[key1][...,np.newaxis]], axis=-1)
    lyr2 = np.concatenate([x_grid_all[...,np.newaxis],
                        retinal_layers[key2][...,np.newaxis]], axis=-1)
    lyr12_xy_all = np.concatenate([lyr1[:,np.newaxis], lyr2[:,np.newaxis]], axis=1)
    layer_pairwise[key] = [utils.remove_nans(tr) for tr in lyr12_xy_all]   

### Ensuring all inner retinal layer segmentations exist for downstream Ppole volume feature measurement

In [ ]:
# Check to make sure all B-scans have inner retinal layer segmentations. If not,
# only return ILM and BM layers - this does not apply for peripapillary scans, where
# only three layers are segmented.
if N_rlayers > 2 and bscan_type == "Ppole":    
    check_layer = "ILM_RNFL"
    check_segs = [trace.shape[1]==0 for trace in layer_pairwise[check_layer]]
    N_missing = int(np.sum(check_segs))
    if N_missing > 0:
        msg = f"WARNING: Found {N_rlayers} retinal layer segmentations, but {N_missing}/{N_scans} B-scans have not been fully segmented for all retinal layers."
        logging.append(msg)
        if verbose:
            print(msg)    
    if N_missing > N_scans//2:
        msg = f"""Over half of the B-scans are missing inner retinal layer segmentations.
Falling back to default state of only analysing whole retina, and removing inner retinal layers in output.
Please segment inner retinal layers for remaining scans to analyse all retinal layers."""
        logging.append(msg)
        if verbose:
            print(msg)
        newlayer_pairwise = {}
        newlayer_pairwise["ILM_BM"] = layer_pairwise["ILM_BM"]
        layer_pairwise = newlayer_pairwise
        N_rlayers = 2        
msg = f"Found {N_rlayers} valid retinal layer segmentations for all B-scans."
logging.append(msg)
if verbose:
    print(msg)

### Work out B-scan acquisition location(s) on en face SLO

In [ ]:
# Construct slo-acquisition image and extract quality of B-scan    
msg = "Accessing IR-SLO and organising metadata..."
logging.append(msg)
if verbose:
    print(msg)
all_mm_points = []
all_quality = []
for m in bscan_meta:
    all_quality.append(m["quality"])
    st = m["start_pos"]
    en = m["end_pos"]
    point = np.array([st, en])
    all_mm_points.append(point)

# Only relevant for Ppole data
quality_mu = np.mean(all_quality)
quality_sig = np.std(all_quality)

# Convert start and end B-scan locations from mm to pixel
all_px_points = []
for point in all_mm_points:
    all_px_points.append(slo_N * point / slo_metadict["field_of_view_mm"])
all_px_points = np.array(all_px_points)

# Python indexing versus .vol all_px_points indexing
all_px_points[:,1,0] -= 1

# Create a copy of slo_acq larger than the SLO to contain all acquisition locations
slo_acq = np.concatenate(3*[slo[...,np.newaxis]], axis=-1)
slo_acq_fixed = slo_acq.copy()
slo_minmax_x = all_px_points[:,:,0].min(), all_px_points[:,:,0].max()
slo_minmax_y = all_px_points[:,:,1].min(), all_px_points[:,:,1].max()

# Work out padding dimensions to ensure the entire fovea-centred acquisition line fits onto slo_fov_max
pad_y = int(np.ceil(abs(min(0,slo_minmax_y[0])))), int(np.ceil(abs(max(0,slo_minmax_y[1]-slo_N))))
pad_x = int(np.ceil(abs(min(0,slo_minmax_x[0])))), int(np.ceil(abs(max(0,slo_minmax_x[1]-slo_N))))
slo_acq = np.pad(slo_acq, (pad_y, pad_x, (0,0)), mode='constant')

In [ ]:
# For peripapillary scans, we draw a circular ROI
acq_palette = np.array(plt.get_cmap("nipy_spectral")(np.linspace(0.1, 0.9, N_scans)))[...,:-1]
if bscan_type == "Peripapillary":
    peripapillary_coords = all_px_points[0].astype(int)
    
    OD_edge, OD_center = peripapillary_coords
    circular_radius = np.abs(OD_center[0] - OD_edge[0])
    circular_mask = grid.create_circular_mask(img_shape=(N,N), 
                                 center=OD_center, 
                                 radius=circular_radius)
    circular_bnd_mask = segmentation.find_boundaries(circular_mask)
    slo_acq[circular_bnd_mask,:] = 0
    slo_acq[circular_bnd_mask,1] = 1
    slo_metadict["stxy_coord"] = f"{OD_edge[0]},{OD_edge[1]}"
    slo_metadict["acquisition_radius_px"] = circular_radius
    slo_metadict["acquisition_radius_mm"] = np.round(circular_radius*slo_metadict["scale_x"],2)
    slo_metadict["acquisition_optic_disc_center_x"] = OD_center[0]
    slo_metadict["acquisition_optic_disc_center_y"] = OD_center[1]

else:
    # Colour palette for acquisition lines, helpful for Ppole map registration onto SLO
    # Use green for single-line scans
    if N_scans == 1:
        acq_palette = [np.array([0,1,0])]

    # Use a spectrum of colour for Ppole/radial scans 
    else:
        acq_palette = np.array(plt.get_cmap("nipy_spectral")(np.linspace(0.1, 0.9, N_scans)))[...,:-1]

    # Loop across acquisition line endpoints, draw lines on SLO
    for idx, point in enumerate(all_px_points):
        loc_colour = acq_palette[idx] #np.array([0,1,0])
        if bscan_type == 'Radial':
            x_idx, y_idx = [[1,0], [0,1]][idx == N_scans//2]  
        else:
            x_idx, y_idx = [[1,0], [0,1]][bscan_type != "V-line"]
        X, y = point[:,x_idx].reshape(-1,1).astype(int), point[:,y_idx].astype(int)
        linmod = LinearRegression().fit(X, y)
        x_grid = np.linspace(X[0,0], X[1,0], 800).astype(int)
        y_grid = linmod.predict(x_grid.reshape(-1,1)).astype(int)
        for (x,y) in zip(x_grid, y_grid):
            if bscan_type == 'Radial':
                x_idx, y_idx = [[x,y], [y,x]][idx == N_scans//2]  
            else:
                x_idx, y_idx = [[y,x], [x,y]][bscan_type != "V-line"]

            # Overlay acquisition locations
            if (0 <= x_idx < slo_N) & (0 <= y_idx < slo_N):
                slo_acq_fixed[y_idx, x_idx] = loc_colour
            slo_acq[pad_y[0]+y_idx, pad_x[0]+x_idx] = loc_colour
        

In [ ]:
# Work out region of interest (ROI) captured by B-scan, helps determine maximum ROI to measure
if scan_type != "Peripapillary":

    # For macular scans, use fovea-centred scan endpoints to work out acquistion ROI
    ROI_pts = all_px_points[fovea_slice_num]
    ROI_xy = np.abs(np.diff(ROI_pts, axis=0)) * np.array([scale_x, scale_x])
    ROI_mm = np.sqrt(np.square(ROI_xy).sum())

else:
    # For peripapillary scans, use circumference of circular acquisition location using
    # OD centre and acquisition circular edge (forming a radius)
    ROI_mm = 2*np.pi*np.abs(np.diff(all_mm_points[0][:,0]))[0]

### Organise metadata

In [ ]:
# Create DataFrame of metadata
bscan_metadict = {}
bscan_metadict["Filename"] = fname
if eye  == 'OD':
    bscan_metadict["eye"] = 'Right'
elif eye == 'OS':
    bscan_metadict["eye"] = 'Left'
bscan_metadict["bscan_type"] = bscan_type
bscan_metadict["bscan_resolution_x"] = N
bscan_metadict["bscan_resolution_y"] = M
bscan_metadict["bscan_scale_z"] = 1e3*scale_z
bscan_metadict["bscan_scale_x"] = 1e3*scale_x
bscan_metadict["bscan_scale_y"] = 1e3*scale_y
bscan_metadict["bscan_ROI_mm"] = ROI_mm
bscan_metadict["scale_units"] = "microns_per_pixel"
bscan_metadict["avg_quality"] = quality_mu
bscan_metadict["retinal_layers_N"] = N_rlayers

# Remove duplicates: store scales as microns-per-pixel, laterality=eye
slo_metadict["slo_scale_xy"] = 1e3*slo_metadict["scale_x"]
for key in ["laterality", "scale_x", "scale_y", "scale_unit"]:
    del slo_metadict[key]
slo_metadict["location"] = scan_type.lower()
slo_metadict["slo_modality"] = slo_metadict.pop("modality")
slo_metadict["field_size_degrees"] = slo_metadict.pop("field_size")
    
# Combine metadata and return with data
metadata = {**bscan_metadict, **slo_metadict}
msg = "Done!"
logging.append(msg)
if verbose:
    print(msg)

# Run OCTolyzer's load function from scratch on this file to check whether it works properly before/after debugging. 

In [ ]:
from importlib import reload
reload(utils)

In [ ]:
bscans, meta, slo, ret, log = utils.load_volfile(vol_path)